In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
pd.options.display.max_columns = 20
np.random.seed(2023)
np.set_printoptions(precision=4, suppress=True)
import matplotlib.pyplot as plt
import matplotlib
plt.rc('figure', figsize=(8, 4))

In [2]:
!wget -q https://raw.githubusercontent.com/kairess/toy-datasets/master/tn_travel_%E1%84%8B%E1%85%A7%E1%84%92%E1%85%A2%E1%86%BC_A.csv
!wget -q https://raw.githubusercontent.com/kairess/toy-datasets/master/tn_traveller_master_%E1%84%8B%E1%85%A7%E1%84%92%E1%85%A2%E1%86%BC%E1%84%80%E1%85%A2%E1%86%A8%20Master_A.csv
!wget -q https://raw.githubusercontent.com/kairess/toy-datasets/master/tn_visit_area_info_%E1%84%87%E1%85%A1%E1%86%BC%E1%84%86%E1%85%AE%E1%86%AB%E1%84%8C%E1%85%B5%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%87%E1%85%A9_A.csv

In [ ]:
# 데이터셋 로드
df_place = pd.read_csv('/content/tn_visit_area_info_방문지정보_A.csv')
df_place.head()

In [ ]:
df_travel = pd.read_csv('/content/tn_travel_여행_A.csv')
df_travel.head()

In [ ]:
df_traveler = pd.read_csv('/content/tn_traveller_master_여행객 Master_A.csv')
df_traveler.head()

In [ ]:
# 데이터 병합
df = pd.merge(df_place, df_travel, on='TRAVEL_ID', how='left')
df = pd.merge(df, df_traveler, on='TRAVELER_ID', how='left')
df.head()

In [ ]:
df[df['TRAVEL_ID'] == 'a_a015688']

In [ ]:
# 전처리 1
df_fil = df[~df['TRAVEL_MISSION_CHECK'].isnull()].copy()
df_fil.loc[:, 'TRAVEL_MISSION_INT'] = df_fil['TRAVEL_MISSION_CHECK'].str.split(';').str[0].astype(int)
df_fil #23898 rows × 68 columns

In [ ]:
# 전처리 2
df_fil = df_fil[[
    'GENDER', 'AGE_GRP',
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1',
    'TRAVEL_COMPANIONS_NUM',
    'TRAVEL_MISSION_INT',
    'VISIT_AREA_NM',
    'DGSTFN',
]]

df_fil = df_fil.dropna()
df_fil # 14930 rows × 15 columns

In [ ]:
# 전처리 3
cat_features_names = [
    'GENDER',
    #'AGE_GRP',
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1',
    #'TRAVEL_COMPANIONS_NUM',
    'TRAVEL_MISSION_INT',
    'VISIT_AREA_NM',
    #'DGSTFN',
]
df_fil[cat_features_names[1:-1]] = df_fil[cat_features_names[1:-1]].astype(int)
df_fil

In [ ]:
# 데이터셋 분류
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df_fil, test_size = 0.2, random_state=2023)
print(train_data.shape)
print(test_data.shape)

In [31]:
!pip install -q catboost # 범주 분류할때 1티어

In [35]:
# Catboost
from catboost import CatBoostRegressor, Pool
train_pool = Pool(train_data.drop(['DGSTFN'], axis=1),# 열방향
                  label = train_data['DGSTFN'],
                  cat_features = cat_features_names)

test_pool = Pool(test_data.drop(['DGSTFN'], 1),# 열방향
                 label = test_data['DGSTFN'],
                 cat_features = cat_features_names)

<ipython-input-35-1ea6618f135b>:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  test_pool = Pool(test_data.drop(['DGSTFN'], 1),# 열방향


In [ ]:
# 모델 생성
model = CatBoostRegressor(
    loss_function = 'RMSE', # 평균제곱근오차
    eval_metric='MAE', # 평균오차
    task_type='CPU',
    depth=6,
    learning_rate=0.01,
    n_estimators=2000)

model.fit(
    train_pool,
    eval_set = test_pool,
    verbose=500,
    plot=True )

In [ ]:
test_data.iloc[2]

In [45]:
# 모델 테스트
model.predict(test_data.iloc[2].drop(['DGSTFN']))

4.427927312087277

In [ ]:
# 테스트데이터 갯수 확인 (1~5점)
test_data[test_data['DGSTFN'] == 3].index

In [ ]:
model.get_feature_importance(prettified=True)

In [ ]:
area_names = df_fil[['VISIT_AREA_NM']].drop_duplicates()
area_names

In [96]:
traveler = {
    'GENDER':'남',
    'AGE_GRP' : 30.0,
    'TRAVEL_STYL_1' : 1,
    'TRAVEL_STYL_2' : 1,
    'TRAVEL_STYL_3' : 1,
    'TRAVEL_STYL_4' : 4,
    'TRAVEL_STYL_5' : 1,
    'TRAVEL_STYL_6' : 4,
    'TRAVEL_STYL_7' : 2,
    'TRAVEL_STYL_8' : 6,
    'TRAVEL_MOTIVE_1' : 3,
    'TRAVEL_COMPANIONS_NUM' : 4.0,
    'TRAVEL_MISSION_INT' : 5 }
results = pd.DataFrame([], columns=['AREA', 'SCORE'])
for area in area_names['VISIT_AREA_NM']:
  input = list(traveler.values())
  input.append(area)
  score = model.predict(input)
  results = pd.concat([results, pd.DataFrame([[area,score]], columns=['AREA','SCORE'])])
results.sort_values('SCORE', ascending=False)[:10]

,AREA,SCORE
0,소피텔 앰배서더 서울,4.446566
0,리빙파워센터,4.421979
0,용산역,4.418065
0,덕수궁,4.417678
0,수서역,4.412488
0,창덕궁,4.410034
0,국립중앙박물관,4.409975
0,송도센트럴파크,4.409758
0,스타필드 하남,4.406912
0,창경궁,4.404291


In [97]:
results.sort_values('SCORE', ascending=True)[:10]

,AREA,SCORE
0,브런치빈 송도직영점,3.972319
0,맥도날드 종로3점,3.972319
0,스파오 라붐서울대점,3.972319
0,월미문화의거리역,3.972319
0,양평물맑은시장,3.982608
0,석계역한일노벨리아시티오피스텔,3.993892
0,맷돌로만 장위점,3.993892
0,메가MGC커피 남양주2청사점,3.993892
0,대갈장군찜,3.993892
0,성수 완당 의정부점,3.993892


In [ ]:
# TSY
tsy = pd.read_csv('/content/tc_codeb_코드B.csv')
tsy = tsy[173:179]
tsy

In [ ]:
# TMT
tmt = pd.read_csv('/content/tc_codeb_코드B.csv')
tmt = tmt[180:190]
tmt

In [ ]:
# MIS
mis = pd.read_csv('/content/tc_codeb_코드B.csv')
mis = mis[105:126]
mis